# Poems Topic Extraction Notebook

This notebook has been used in order to extact measure semantic similarity among poems.

In [1]:
import os
import numpy as np
import unidecode
from tqdm import tqdm
import pickle
import re
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Converts a poem with multiple lines into one singular string

def poem_to_string(poem):
    result = ""
    for i in poem:
        result +=' '+i
    return result

In [3]:
# Iterates over all the nawbas and appends the poems into two lists (original and transliterated versions)

total_list_poems = []
total_list_poems_transliterated = []

test_dict = {}

for filename in os.listdir('outputs/'):
    if filename.endswith('.txt'):
        name = os.path.join('/Users/Morgan/Desktop/UPF/musicology.nosync/outputs', filename)
        with open(name, "rb") as fp:  

            b = pickle.load(fp)
        

        
    for i in range(len(b)):

        if len(b[i])>4 and b[i][-1]=='qaṣīdah':
            #print(filename.split('.txt')[0])
            
            #print(b[i][0])
        #if len(b[i])>4:
        #    if not isinstance(b[i][4], float):
            
            total_list_poems.append(poem_to_string(b[i][4]))
            total_list_poems_transliterated.append(poem_to_string(b[i][5]))
            test_dict[(filename.split('.txt')[0],b[i][0])] = (len(total_list_poems)-1, [])
            
            
            
print('Original size :',len(total_list_poems))
print('Transliterated size :',len(total_list_poems_transliterated))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/Morgan/Desktop/UPF/musicology.nosync/outputs/9b194c42-f974-42f8-89b1-123bb07170b8.txt'

In [4]:
# For both versions, lists all the keywords and builds a vector representation of each poem
# Each component of these vectors corresponds to the presence of each keyword in the corresponding poem 


vectorizer = CountVectorizer(analyzer='word',
                             max_df=0.95, 
                             min_df=0.04,
                             ngram_range=(1,6))

X_original = vectorizer.fit_transform(total_list_poems)

print('Number of keywords original :', len(vectorizer.get_feature_names()))
X_array_original = X_original.toarray()
print(len(np.where(X_array_original==0)[0])/(X_array_original.shape[0]*X_array_original.shape[1])*100,'%')


vectorizer = CountVectorizer(analyzer='word',
                             max_df=0.95,
                             min_df=0.04,
                             lowercase=True,
                             strip_accents='unicode',
                             ngram_range=(1,6))

X = vectorizer.fit_transform(total_list_poems_transliterated)

print('Number of keywords transliterated :', len(vectorizer.get_feature_names()))
X_array = X.toarray()
print(100-np.count_nonzero(X_array)/(X_array.shape[0]*X_array.shape[1])*100,'%')

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [131]:
# For a given poem, finds the closest ones based on the vector representation

def neighbor_graph(array, index):
    neighbors = []
    instance = array[index]
    for i in tqdm(range(len(array))):
        if i!=index:
            # only looks at non-zero components in common from both vectors
            indexes = list(set(np.nonzero(array[i])[0]).intersection(np.nonzero(instance)[0]))
            neighbors.append(len(indexes))
    return np.array(neighbors)

In [132]:
# Code to test the function built above, i denotes the index of the poem (from 0 to 147)
# nb denotes the number of similar poems that are returned

i = 25
nb = 5

neighbors = neighbor_graph(X_array, i)
neighbors_original = neighbor_graph(X_array_original, i)


n = np.argsort(neighbors)[-nb:]
n_original = np.argsort(neighbors_original)[-nb:]

print('Poem = ',i,total_list_poems[i])
print('Closest elements :',n)
for j in n:
    if j == i:
        continue

    print('Score {}'.format(j),':',neighbors[j])
for j in n:
    if j == i:
        continue

    print(total_list_poems[j])
    print('-----------------')
    
    
    
print('\n')
print('Original based :\n')



print('Poem = ',i,total_list_poems[i])
print('\n')
print('Closest elements :',n_original)
for j in n_original:
    if j == i:
        continue

    print('Score {}'.format(j),':',neighbors_original[j])
for j in n_original:
    if j == i:
        continue

    print(total_list_poems[j])
    print('-----------------')    

100%|██████████| 153/153 [00:00<00:00, 36490.87it/s]

Poem =  25  وَمَنْ تَكُنْ بِرَسُولِ اللَّهِ نُصْرَتُهُ إِنْ تَلْقَهُ الأُسْدُ فِي آجَامِهَا تَجِمِ
Closest elements : [ 59  60 119  92   6]
Score 59 : 3
Score 60 : 3
Score 119 : 4
Score 92 : 4
Score 6 : 4
 جُفُونِي قَادَتْ إِلَى حَيْنِي وَثَارِي عِنْدِي فَمَا أَطْلُبْ دَعُونِي أَقْتَصُّ مِنْ جَفْنِي بِسُهْدِي وَعَبْرَتِي أَسْكُبْ
-----------------

-----------------
 أَبْشِرْ لَقَدْ نِلْتَ مَا تَرْجُو وَتَنْتَظِرُ وَقَدْ جَرَى بِالَّذِي تَخْتَارُهُ القَدَرُ وَسَاعَدَتْكَ مِنَ الأَيَّامِ أَرْبَعَةٌ العِزُّ وَالنَّصْرُ وَالتَّمْكِينُ   وَالظَّفَرُ
-----------------
 أَيَا فَاضِحَ البَدْرِ عِنْدَ التَّمَامْ فَهَلْ لَكَ وَاوٌ وَصَادٌ وَلامْ وَهَلْ لا عَطَفْتَ عَلَى عَاشِقٍ يَهِيمُ بِقَافٍ وَبَاءٍ وَلامْ تَمَلَّكْتَ عَقْلِي بِثَغْرٍ شَنِيبْ وَرِيقٍ كَعَيْنٍ وَسِينٍ وَلامْ
-----------------
 وَمَنْ تَكُنْ بِرَسُولِ اللَّهِ نُصْرَتُهُ إِنْ تَلْقَهُ الأُسْدُ فِي آجَامِهَا تَجِمِ
-----------------


Original based :

Poem =  25  وَمَنْ تَكُنْ بِرَسُولِ اللَّهِ نُصْرَتُهُ إِنْ تَلْقَهُ الأُسْدُ 

In [134]:
import random

# function that returns a random set of dissimilar poems

def get_dissimilar(array, index, nb):
    anti_neighbors = []
    neighbors = neighbor_graph(array, index)
    for i in range(nb):
        j =  index
        while (j in neighbors or j==index) or array[index, j]>0:
            j = random.randint(0,array.shape[1]-1)
            print(j)
        anti_neighbors.append(j)
    return anti_neighbors

In [139]:
# We carry out this process for all the poems:
# Build a list of n semantically similar poems
# Build a list of n semantically dissimilar poems


for i in range(len(total_list_poems)):
    for j in test_dict.keys():
        if test_dict[j][0]==i:
            
            #neighbors = neighbor_graph(X_array, i)
            #neighbors_original = neighbor_graph(X_array_original, i)
            #n = np.argsort(neighbors)[-nb:]
            #n_original = np.argsort(neighbors_original)[-nb:]
            n = get_dissimilar(X_array, i, 5)
            #test_dict[j][1].append(n_original)
            test_dict[j][1].append(n)          
            
            
# final processing of the dictionary to ease its future use
new_dict = {}
for key in list(test_dict.keys()):
    temp = []
    for neighbor_index in test_dict[key][1][0]:
        for key_2 in list(test_dict.keys()):
            if test_dict[key_2][0] == neighbor_index:
                temp.append(key_2)
    new_dict[key] = temp
    
# we save the dictionary in a txt file 
with open("dissimilarity_transliterated.txt", "wb") as fp:   #Pickling
    pickle.dump(new_dict, fp)

  0%|          | 0/153 [00:00<?, ?it/s]

108
64
69
189
37
37
9
175
54
20
124
87
136
39
86
11
101
145
176
141
121
50
81
66
160
158
182
183
130
113
147
76
171
90
185
62
24
120
65
3
157
98
62
38
150
154
3
26
55
61
49
156
34
104
49
149
34
13
38
179
187
37
44
86
117
174
48
101
89
102
31
76
129
87
16
159
64
90
165
74
82
3
121
21
147
58
40
115
189
170
49
190
52
124
6
14
57
132
147


  0%|          | 0/153 [00:00<?, ?it/s]

83
67
9
145
88
50
152
158
180
162
13
187
40
160
52
51
180
39
158
29
157
46
155
59
19
54
147
135
7
47
13
124
131
65
18
74
162
115
56
71
82
182
172
72
62
34
92
154
190
121
123
67
71
17
128
51
64
44
70
88
191
34
180
73
190
172
118
138
45
76
74
1
17
188
189
76
82
180
190
1
181
46
183
131
189
41
100
92
36
6
18
66
174
31
24
137
9
7
108
21
177
79
113
115
24
61
108
59
96
92
44
48
137
26
50
100


100%|██████████| 153/153 [00:00<00:00, 78422.16it/s]

...

122
59
27
83
145
152
74
12
141
51
18
73
87
158
110
22
96
3
183
91
151
7
174
95
190
120
173
75
169
14
105
117
153
60
16
99
53
76
21
149
172
103
3
165
117
159
179
173
136
185
46
126
115
103
187
79
53
118
135
43
66
54
139
65
124
87
59
36
189
20
166
56
163
169
88
116
17
74
105
188
23
89
76
11
8
131
16
119
94
38
38


  0%|          | 0/153 [00:00<?, ?it/s]

129
165
81
56
80
28
114
8
94
5
13
153
170
141
84
146
113
22
52
95
121
38
104
80
184
162
75
15
0
152
37
151
13
28
156
179
102
136
44
172
122
92
50
134
6
185
73
64
35
123
14
137
110
168
83
167
9
90
161
109
48
45
36
16
175
20
75
0
48
159
163
7
149
91
100
170
13
87
153
7
164
35
116
178
68
162
84
15
83
156
104
82
117


100%|██████████| 153/153 [00:00<00:00, 55036.75it/s]

15
137
107
36
88
168
15
82
11
91
148
127
155
9
81
148
7
191
115
66
144
142
43
14
57
38
123
64
100
93
5
